<a href="https://colab.research.google.com/github/ShahryarAkbar/NLP-Project-/blob/main/CODE%20GPT_2_Dialogue_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 69.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 106.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 31.4 MB/s eta 0:00:00


In [ ]:
import torch
from torch.utils.data import Dataset
from transformers import GPT2LMHeadModel, GPT2Tokenizer, DataCollatorForLanguageModeling, Trainer, TrainingArguments
import pandas as pd
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print(device)

# Load and preprocess your dataset
dataset = pd.read_csv('/content/drive/MyDrive/Dialouge generation.csv')  # Adjust the file path as per your dataset location

# Extract customer's utterances and agent's responses
customer_utterances = dataset['Situation'].tolist()
agent_responses = dataset['empathetic_dialogues'].tolist()

# Convert emotion/sentiment labels to numerical format if required
labels = dataset['labels'].tolist()

# Load the tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Manually add a pad token to the tokenizer
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# Tokenize your dataset
def tokenize_dataset(customer_utterances, agent_responses):
    tokenized_lines = []
    for customer, agent in zip(customer_utterances, agent_responses):
        tokenized_line = tokenizer.encode(customer + agent)
        tokenized_lines.append(tokenized_line)

    return tokenized_lines

tokenized_dataset = tokenize_dataset(customer_utterances, agent_responses)

# Define a custom dataset class
class CustomDataset(Dataset):
    def __init__(self, tokenized_dataset):
        self.tokenized_dataset = tokenized_dataset

    def __len__(self):
        return len(self.tokenized_dataset)

    def __getitem__(self, idx):
        return torch.tensor(self.tokenized_dataset[idx])

# Create an instance of the custom dataset
train_dataset = CustomDataset(tokenized_dataset)

# Define the training arguments and trainer
training_args = TrainingArguments(
    output_dir='output_directory',
    overwrite_output_dir=True,
    num_train_epochs=1,  # Adjust the number of epochs according to your requirements
    per_device_train_batch_size=1,  # Adjust batch size according to your resources
    save_steps=1000,
    save_total_limit=2,
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)

# Start the fine-tuning process
trainer.train()


cuda


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,3.070000
1000,2.896600
1500,2.830600
2000,2.803600
2500,2.698300
3000,2.706800
3500,2.738600
4000,2.801100
4500,2.762700
5000,2.738300


TrainOutput(global_step=64636, training_loss=2.4324902455931703, metrics={'train_runtime': 8601.9832, 'train_samples_per_second': 7.514, 'train_steps_per_second': 7.514, 'total_flos': 1381939364736000.0, 'train_loss': 2.4324902455931703, 'epoch': 1.0})

In [ ]:
import pandas as pd

# Load the dataset
dataset = pd.read_csv('/content/drive/MyDrive/Dialouge generation.csv')

# Check for missing values
missing_values = dataset.isnull().sum()
print(missing_values)


Unnamed: 0                  0
Situation                   0
emotion                     4
empathetic_dialogues        0
labels                      0
Unnamed: 5              64523
Unnamed: 6              64631
dtype: int64


In [ ]:
# Remove rows with missing values
dataset = dataset.dropna()


In [ ]:
# Specify the directory path to save the trained model and tokenizer
output_dir = '/content/drive/MyDrive/trained_model/'

# Create the directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Save the trained model and tokenizer
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)


('/content/drive/MyDrive/trained_model/tokenizer_config.json',
 '/content/drive/MyDrive/trained_model/special_tokens_map.json',
 '/content/drive/MyDrive/trained_model/vocab.json',
 '/content/drive/MyDrive/trained_model/merges.txt',
 '/content/drive/MyDrive/trained_model/added_tokens.json')

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

model = GPT2LMHeadModel.from_pretrained('/content/drive/MyDrive/trained_model/')
tokenizer = GPT2Tokenizer.from_pretrained('/content/drive/MyDrive/trained_model/')


In [ ]:
model.eval()


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
# Example input prompt
input_prompt = "are you friends with"

# Tokenize the input prompt
input_ids = tokenizer.encode(input_prompt, return_tensors='pt')

# Generate dialogues using beam search with beam width of 1
output = model.generate(
    input_ids,
    max_length=50,
    num_beams=1,
    no_repeat_ngram_size=2,
    early_stopping=True,
    num_return_sequences=1,
    pad_token_id=tokenizer.eos_token_id,  # Set pad token id to eos token id
    attention_mask=input_ids.ne(tokenizer.pad_token_id)  # Set attention mask
)

# Decode the generated tokens back into text
generated_dialogues = [tokenizer.decode(dialogue, skip_special_tokens=True) for dialogue in output]

# Use a set to store unique responses
unique_dialogues = set()

# Remove "Customer" and "Agent" tokens and trim the dialogues
for dialogue in generated_dialogues:
    sentences = dialogue.split(":")  # Split the dialogue into sentences
    cleaned_sentences = [sentence.strip() for sentence in sentences]  # Remove leading/trailing whitespaces
    cleaned_dialogue = " : ".join(cleaned_sentences)  # Reconstruct the cleaned dialogue
    unique_dialogues.add(cleaned_dialogue)

# Print the unique generated dialogues
for dialogue in unique_dialogues:
    print(dialogue)


are you friends with someone who is not your friend?
Agent : I am not sure. I am sure that you are not friends.
 agent : That is a good idea. 
 Agent : It is. It is hard to be friends
